In [2]:
from pathlib import Path
import gin
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2023-12-22 15:24:29.825 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 2920.36it/s]


In [4]:
len(train), len(valid)

(81, 20)

In [4]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([32, 30, 3]), torch.Size([32]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [5]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.GIN, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs=None
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.GIN: 1>, <ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: None

In [7]:
gin.parse_config_file("gestures.gin")
model = rnn_models.BaseRNN()

In [8]:
gin.get_bindings("BaseRNN")

{'input_size': 3, 'hidden_size': 128, 'num_layers': 3, 'horizon': 20}

Test the model. What is the output shape you need? Remember, we are doing classification!

In [9]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [10]:
accuracy(y, yhat)

tensor(0.0625)

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [11]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [12]:
yhat[0]

tensor([-0.1400, -0.0424,  0.0504, -0.0596, -0.0328, -0.0273, -0.0510, -0.0780,
        -0.0487, -0.1507, -0.1819, -0.1738,  0.0707, -0.0735, -0.1079,  0.0287,
         0.0289,  0.0847,  0.2338,  0.1579], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(3.0064, grad_fn=<NllLossBackward0>)

In [14]:
gin.get_bindings("GRUmodel")

{}

In [15]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")


<Experiment: artifact_location='/home/azureuser/code/ADS-DeepLearning-project/notebooks/3_recurrent_networks/mlruns/1', creation_time=1703253907365, experiment_id='1', last_update_time=1703253907365, lifecycle_stage='active', name='gestures', tags={}>

In [16]:
modeldir = Path("../../models/gestures/").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

In [21]:
from datetime import datetime

gin.parse_config_file("gestures_gru.gin")

with mlflow.start_run():
    mlflow.set_tag("model", "GRUmodel-9")
    mlflow.set_tag("dev", "tom")
    mlflow.log_params(gin.get_bindings("GRUmodel"))

    model = rnn_models.GRUmodel()

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau
    )
    trainer.loop()

    tag = datetime.now().strftime("%Y%m%d-%H%M")
    modelpath = modeldir / (tag + "model.pt")
    torch.save(model, modelpath)
    
    mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")

2023-12-22 15:47:58.426 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to gestures/20231222-154758
100%|██████████| 81/81 [00:05<00:00, 13.68it/s]
2023-12-22 15:48:04.982 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 2.4861 test 2.1278 metric ['0.2406']
100%|██████████| 81/81 [00:05<00:00, 13.91it/s]
2023-12-22 15:48:11.425 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 1.7976 test 1.5968 metric ['0.3594']
100%|██████████| 81/81 [00:05<00:00, 13.83it/s]
2023-12-22 15:48:17.907 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 1.3623 test 1.1978 metric ['0.5125']
100%|██████████| 81/81 [00:05<00:00, 13.93it/s]
2023-12-22 15:48:24.361 | INFO     | mltrainer.trainer:report:171 - Epoch 3 train 1.0643 test 0.8733 metric ['0.6875']
100%|██████████| 81/81 [00:06<00:00, 13.47it/s]
2023-12-22 15:48:31.033 | INFO     | mltrainer.trainer:report:171 - Epoch 4 train 0.7242 test 0.6833 metric ['0.7406']
100%|██████████| 81/81 [00:06<00:00, 13.23it/s]
202

Try to update the code above with the following two commands.
    
```python
gin.parse_config_file('gestures_gru.gin')
model = rnn_model.GRUmodel()
```

To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [18]:
mlflow.end_run()

Excercises:

- improve the RNN model
- test different things. What works? What does not?
- experiment with either GRU or LSTM layers, create your own models + ginfiles. 
- experiment with adding Conv1D layers.

You should be able to get above 90% accuracy with the dataset.